In [1]:
!pip install langchain
!pip install sentencepiece
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -U sentence-transformers
!pip install -U langchain-community
!pip install bitsandbytes
!pip install -U faiss-cpu langchain-openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.to

In [ ]:
# load required library
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
import pandas as pd
from langchain.chains import LLMChain
from langchain_community.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough

from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
import os
import random
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough


from huggingface_hub import login
login(token='hugging_face_key')

quantization_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_compute_dtype=torch.bfloat16
)

model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map='auto', quantization_config=quantization_config)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000)
llm = HuggingFacePipeline(pipeline=pipe)

def create_dataframe_from_csv(file_path):
    loader = CSVLoader(file_path=file_path, encoding="latin-1")
    data = loader.load()
    return data

# Specify the path to your single CSV file
file_path = "RAG_LLM_Train_Dataset.csv"
dataframe = create_dataframe_from_csv(file_path)
splitter = RecursiveCharacterTextSplitter(chunk_size=4000,chunk_overlap=50)
split_documents = splitter.split_documents(dataframe)

db1 = FAISS.from_documents(split_documents, embeddings)

import re
def response_creation(question_file, output_file, llm, PROMPT, retriever):
    # Create LLMChain with output parser
    llm_chain = LLMChain(llm=llm, prompt=PROMPT)
    df = pd.read_csv(question_file)

    # Ensure the input CSV has a column named "description"
    if "description" not in df.columns:
        raise ValueError("Input CSV must contain a 'description' column.")

    # Add columns for responses
    df["LLM_Tactics"] = ""
    df["LLM_Techniques"] = ""
    df["LLM_Full_Response"] = ""

    # Open the output file to save incrementally
    with open(output_file, mode="w", newline="") as file:
        # Write the header row to the file
        df.head(0).to_csv(file, index=False)

        # Iterate over the rows and get responses
        for idx, row in df.iterrows():
            question = row["description"]

            try:
                # Call invoke and get clean output
                rag_chain = (
                    {"context": retriever, "question": RunnablePassthrough()}
                    | llm_chain
                )
                response = rag_chain.invoke(question)['text']

                # Extract and process response
                answer = response.split("[/INST]")[-1].strip()

                # Extract Tactics and Techniques using regex
                tactics = ""
                techniques = ""

                # Check the first line explicitly
                response_lines = response.split("\n")
                first_line = response_lines[0].strip()
                if first_line.startswith(("Tactic:", "Tactic(s):", "Tactis:", "Tactic ID:")):
                    tactics = line.split(":", 1)[1].strip()

                # Process the rest of the lines
                for line in response_lines:
                    if line.startswith(("Tactic:", "Tactic(s):", "Tactis:", "Tactic ID:")):
                        tactics = line.split(":", 1)[1].strip()
                    elif line.startswith(("Technique:", "Technique(s):", "Techniques:", "Technique ID:")):
                        techniques = line.split(":", 1)[1].strip()

                # Update the DataFrame
                df.at[idx, "LLM_Tactics"] = tactics
                df.at[idx, "LLM_Techniques"] = techniques
                df.at[idx, "LLM_Full_Response"] = answer

                # Write the current row to the file
                df.iloc[[idx]].to_csv(file, header=False, index=False)

            except Exception as e:
                print(f"Error processing row {idx}: {e}")
                df.at[idx, "LLM_Full_Response"] = f"Error: {e}"

    print(f"Responses saved incrementally to {output_file}")


"""# Create PromptTemplate and LLMChain

## Prompt 3: Simple Prompt
"""

qna_prompt_template="""### [INST] Instruction: You are a cybersecurity expert specializing in mobile application security. Your task is to respond to queries associated with Android applications. Each Android app has different static features.

### Understanding Static Features
Static features are characteristics of an Android application that can be analyzed without executing the app. These features provide insights into the app's behavior, permissions, and components.

Here are examples of static features commonly used in analyzing Android applications:

**Permissions**: These are declarations that specify what resources the app can access. , e.g., android.permission.INTERNET, android.permission.ACCESS_FINE_LOCATION.
**Activities**: Activities represent a single screen in an app's user interface., e.g., com.example.app.MainActivity.
**Services**: Services running in the background, e.g., com.example.app.BackgroundSyncService.
**Intents**: Specify how the app communicates with other apps or handles specific actions, e.g., android.intent.action.VIEW.
**Broadcast Receivers**: Components listening for system or app-level broadcasts, e.g., com.example.app.BatteryLowReceiver.
By analyzing the static features, it is possible to infer the tactics and techniques used by adversaries to compromise mobile security. For more details on static features, you can refer to the [Android Developers documentation](https://developer.android.com/guide/topics/manifest/manifest-intro).

To conduct attacks through android applications, adversaries use different tactics and techniques.
### Understanding Tactics
In the context of cybersecurity, **Tactics** refer to the general goals or objectives that an adversary aims to achieve through their actions. These can include a wide range of malicious activities, such as: Collection, Impact, Defense Evasion.

### Understanding Techniques
**Techniques** are specific methods or actions that adversaries use to achieve their tactics. Each tactic in the MITRE ATT&CK framework is mapped to multiple techniques. Some examples of Techniques are: T1636, T1582, T1604, T1437..

More details about Tactics and Techniques are avaialble at **MITRE ATT&CK**(https://attack.mitre.org/matrices/mobile/android/), which provides a matrix that maps tactics to specific techniques used by adversaries. Other threat intelligence platforms such as **VirusTotal**(https://www.virustotal.com/gui/home/upload), **TRAM**(https://github.com/mitre-attack/tram), and **Alienvault** (https://otx.alienvault.com/) also provide valuable insights into potential malicious behaviors and TTPs associated with various applications.

Using the details and context provided, analyze the static features of an Android application and identify the associated tactics and techniques. One app may be associated with multiple tactics and techniques.

Response Format:

Tactic(s): <List of tactics, e.g., Collection, Impact>
Technique(s): <List of techniques, e.g., T1636, T1582, T1604, T1437, T1521, T1417>
If the provided data is insufficient to determine the tactics and techniques, respond with: "Not enough information."

Note: Only provide the tactics and techniques in the specified format. Do not include any additional explanations or comments.

{context}

### Question: {question} [/INST]"""

PROMPT = PromptTemplate(
   template=qna_prompt_template, input_variables=["context", "question"]
)


# Load the retriver
retriever = db1.as_retriever(search_kwargs = {"k" : 25})
# Read questions from the CSV file
question_file = "RAG_LLM_Test_Dataset.csv"  # Path to input CSV file
output_file = "RAG_Top_25_Prompt_3_result.csv"  # Path to save output CSV file
response_creation(question_file, output_file, llm, PROMPT,retriever)


<ipython-input-2-ff9b34cece6b>:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)
<ipython-input-2-ff9b34cece6b>:34: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, c

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-2-ff9b34cece6b>:40: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
<ipython-input-2-ff9b34cece6b>:58: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=PROMPT)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end g

Responses saved incrementally to /content/drive/MyDrive/Dincy_TTP_LLM/RAG/RAG_Top_25_Prompt_3_result.csv
